In [7]:
import pandas as pd
import os
import glob
import re
from collections import Counter

In [9]:
def createDirectory(directory): 
    """ 새로운 폴더를 생성하는 함수 """
    try: 
        if not os.path.exists(directory): 
            os.makedirs(directory) 
    except OSError: 
        print("Error: Failed to create the directory.")

In [2]:
def make_long_format(d):
    """ 데이터프레임을 long format으로 변환하는 함수 """
    
    df = pd.DataFrame.from_dict(d).rename_axis('자치구').reset_index()
    df.fillna(0, inplace = True)
    df = pd.melt(df, id_vars = ['자치구'], var_name = '이름', value_name = '빈도')
    df['빈도'] = df['빈도'].astype('int')
    df.drop(df[df['빈도'] == 0].index, inplace = True)
    df = df.reindex(columns = ['이름', '자치구', '빈도'])
    
    return df 
            

In [3]:
# 서울특별시 자치구 목록. 규모가 작아 리스트 직접 제작. 
borough = ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구',
              '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구',
              '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구',
              '은평구', '종로구', '중구', '중랑구']

In [10]:
location = os.getcwd()

fname_list = os.listdir(os.path.join(location, "회의록_모음"))
file_list = glob.glob(os.path.join(location, "회의록_모음/*"))

# 원하는 폴더 이름 설정.
dir_name = "위원 별 자치구 언급 빈도 분석"
createDirectory(dir_name)

for i in range(len(fname_list)):
    df = pd.read_csv(file_list[i], encoding = 'euc-kr')

    person_dict = {}
    for j in range(len(df)):
        person = df.iloc[j, 7]
        line = df.iloc[j, 9]

        for k in range(len(borough)):
            b = borough[k]
            if b in line:
                cnt = line.count(b)
                if person in person_dict:
                    if b in person_dict[person]:
                        person_dict[person][b] += cnt
                    else:
                        person_dict[person][b] = cnt
                else:
                    person_dict[person] = {b : cnt}

    make_long_format(person_dict).to_csv(os.path.join(dir_name, "%s_자치구.csv" % fname_list[i][:-4]), index = False, encoding = 'euc-kr')
    print("%s_자치구.csv 완료" % fname_list[i][:-4])

서울특별시의회 회의록 제277회[정례회] (2017.11.01 ~ 2017.12.20)_자치구.csv 완료
서울특별시의회 회의록 제284회[정례회] (2018.11.01 ~ 2018.12.20)_자치구.csv 완료
서울특별시의회 회의록 제290회[정례회] (2019.11.01 ~ 2019.12.20)_자치구.csv 완료
서울특별시의회 회의록 제298회[정례회](2020.11.02 ~ 2020.12.22)_자치구.csv 완료
서울특별시의회 회의록 제303회[정례회](2021.11.01 ~ 2021.12.27)_자치구.csv 완료
